In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False
clear_model_results = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import os
import pandas as pd
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df
)
from openavmkit.utilities.settings import (
    load_settings
)
from openavmkit.checkpoint import (
    from_checkpoint,
    read_checkpoint,
    write_checkpoint,
    delete_checkpoints
)
from openavmkit.data import (
    write_canonical_splits
)
from openavmkit.benchmark import (
    run_models, 
    MultiModelResults
)
from openavmkit.utilities.plotting import (
    plot_histogram_df
)
from openavmkit.land import (
    run_land_analysis,
    finalize_land_values
)
from openavmkit.ratio_study import (
    run_and_write_ratio_study_breakdowns
)

%matplotlib inline

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)
settings = load_settings()

In [ ]:
if clear_checkpoints:
    delete_checkpoints("3-model")

# 2. Model

We load the cleaned data from the last checkpoint:

In [ ]:
# load the data
sup = read_checkpoint("2-clean-04-out")

We separate our test set from our training set.  
This guarantees we have one durable source of truth for test/train set.

In [ ]:
write_canonical_splits(
    sup["sales"],
    load_settings()
)

We run all of our modeling code:  
- Run individual models
- Run preliminary ratio studies & horizontal equity studies
- Generate overall utility scores per model
- Generate ensembles of the best models
- Do this three times:
  - main : vacant+improved dataset, predicting full market value
  - hedonic : vacant+improved dataset, predicting land value
  - vacant : vacant dataset, predicting land value

In [ ]:
results = from_checkpoint("3-model-00-run-models", run_models,
    {
        "sup": sup,
        "settings": load_settings(),
        "use_saved_results": (not clear_model_results),
        "verbose": verbose,
        "run_main": True,
        "run_vacant": True
    }
)

# Generate ratio study reports

In [ ]:
# run ratio study reports
run_and_write_ratio_study_breakdowns(load_settings())

# 3. Land Analysis

In [ ]:
run_land_analysis_sup(sup, load_settings(), verbose)

In [ ]:
finalize_land_values_sup(
    sup, 
    load_settings(), 
    True, 
    verbose
)